In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio

%matplotlib inline
pio.renderers.default='iframe'

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

# 1.0 Data retrieval

In [2]:
df = pd.read_pickle("../../../data_lake/output/df_merged.pkl")

# 2.0 EDA

Specify columns format

In [3]:
df = df.astype('object')

In [4]:
num_col = ['TransactionAmt','C1', 'C2', 'C3', 'C4', 'C6', 'C7',
       'C8', 'C10', 'C11', 'C12', 'C13', 'C14','D1', 'D2', 'D3', 'D4', 'D5',
       'D6', 'D7', 'D8', 'D9', 'D10', 'D12', 'D13', 'D14', 'D15']

In [5]:
for i in df:
    if i in num_col:
        df[i] = df[i].astype('float')

Given the "masked" meaning of C# and D# columns, i've decided to remove them by replacing with their max values computed along avery row: some information will be saved anyway in this way.

In [6]:
df['num_accounts_related_to_user'] = df.filter(regex='C').max(axis=1)

/tmp/ipykernel_65/263844738.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [7]:
df = df.drop(columns=['C1', 'C2', 'C3', 'C4', 'C6', 'C7',
       'C8', 'C10', 'C11', 'C12', 'C13', 'C14'])

In [8]:
df['num_days_previous_transaction'] = df.filter(regex='D').max(axis=1)

/tmp/ipykernel_65/3198743402.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [10]:
df = df.drop(columns=['D1', 'D2', 'D3', 'D4', 'D5',
       'D6', 'D7', 'D8', 'D9', 'D10', 'D12', 'D13', 'D14', 'D15'])

Create costumer id column by aggergating data from card, address and email domain

In [12]:
df['customer_id_1'] = df[['card1',
       'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2',
       'P_emaildomain']].apply(lambda x: '{}_{}_{}_{}_{}_{}_{}_{}_{}'.format(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8]), axis=1)

In [14]:
df['customer_id'] = pd.factorize(df['customer_id_1'])[0]

In [15]:
df['customer_id'].nunique()

38897

Create new features from time info

In [16]:
df['num_transaction_per_time'] = df['time_id'].map(df['time_id'].value_counts())

In [17]:
df['multi_transaction_per_time'] =  df['num_transaction_per_time'].apply(lambda x: 1 if x>1 else 0)

In [18]:
df = df.drop(columns='customer_id_1')

In [19]:
df = df.drop(columns='time_id')

# 3.0 Output

In [20]:
df.head(1)

,TransactionID,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist2,P_emaildomain,R_emaildomain,M4,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,num_accounts_related_to_user,num_days_previous_transaction,customer_id,num_transaction_per_time,multi_transaction_per_time
0,2987004,0,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,1.0,0.0,0,1,0


In [21]:
df.to_pickle("../../../data_lake/output_prod/df_under.pkl")
df.to_csv("../../../data_lake/output_prod/df_under.csv")